# 距離センサ3つのみでAI RC CARを実現できるか実験してみる。

## このノートブックの概要
ロボット前方に3つの距離センサを搭載し、AI RC CARと同様に、NNに BCできるかを実験する。

## 事前準備

airc-microリポジトリを利用して、実機にて学習用データを収集する。収集するデータは```左センサ値, 中央センサ値, 右センサ値, 舵角, スロットル値```の形のcsvフォーマットとなる。


## 学習データの準備

学習データはagent.recordにリストとして溜まっている。以下の並びのデータが収集されている。


```[中央センサの値,右センサの値,左センサの値,舵角,速度]```

以下ではPytorchで学習させるためにデータの整形と変換を行う。正規化とかしてないけど多分動く。

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

# Load dataset

uploaded_filename = 'test.csv'
skipping = 50
x = np.loadtxt(uploaded_filename, delimiter=",", skiprows=skipping)


# xを2個の配列に水平分割
train_data, train_labels = np.hsplit(x, [3])
train_labels = np.squeeze(np.hsplit(train_labels, 2)[0])
train_labels = train_labels+1

max_value = train_data.max()
min_value = train_data.min()
train_data = (train_data - min_value) / (max_value-min_value)


print(train_data)
print(train_labels)


## NNのモデル定義

NNのネットワークを構築する。入力3,出力2の全結合3層のニューラルネットワーク、口が裂けても深層学習とか言ってはいけない。入力3はセンサの値をそのまま突っ込む。出力２は速度と舵角が出力される。中間層は16あるけど、多分、さらに半分に落としても問題ない。

In [ ]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential([
    layers.Dense(8, activation='relu', input_shape=(3,)),
    layers.Dense(8, activation='relu'),
    layers.Dense(3, activation="softmax"),
  ])

optimizer = keras.optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
# model.compile(loss='mse',
#               optimizer = optimizer,
#               metrics = ['mae'])
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

## 学習ループ

Tensorflowの学習ループ


In [ ]:
EPOCHS = 100

history = model.fit(
  train_data, train_labels,
  epochs=EPOCHS, validation_split = 0.2, verbose=1)
print(history)

## TensorFlow lite for microcontrollerへモデル変換



In [ ]:
# Convert to TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert() #量子化はせずFP32のまま処理していことに注意!

open("converted_model.tflite", "wb").write(tflite_model)


!apt-get install xxd
!xxd -i converted_model.tflite > airc_model.cpp

## Download FlatBuffer C files.

TensorFlow lite for microcontroller向けに保存したモデルファイルをダウンロードする。ダウンロードされたairc_model.cppはそのままairc-micro/model/model.cppへ上書きする。また。min_val, max_valをmodel.hの同名変数定義へそれぞれ転記する。

In [ ]:
from google.colab import files
files.download('airc_model.cpp')
print("min_val={}, max_val={}".format(min_value,max_value))